**Indices**  
j - job  
m - machine 

**Decision Variable**  
$s_{j,m}$ - start time of job j at machine m  
$e_{j,m}$ - end time of job j at machine m  
$d_{j,m}$ - duration of job j at machine m  
$b_{j,m}$ - machine m is selected for job j     
$a_{j,i,m}$ - job i precedes job j at machine m 

**Parameters**  
H - Horizon  
$d_j$ - delivery date of job j  
$q_j$ - order qty of job j  
$o_{j,m}$ - output per hour of job j at machine m  
$set_{i,j}$ - setup time between job i & j   


**Objective** (Not implemented right now)  
$ \sum_{j} d_j - max(e_{j,m})  \qquad \forall m \in mc$    


**Constraints**  
$ \sum_m e_{j,m} < s_{i,m} or s_{j,m} > e_{i,m} \qquad \forall j,i \in jobs$  
$ \sum_j o_{j,m} \times d_{j,m} >= q_j \qquad \forall m \in ma $    
$ \sum_m s_{i,m} >= e_{j,m} + set_{i,j} \qquad if \quad b_{j,m}=1, b_{i,m}=1, a_{j,i,m}=1 $  

In [1]:
!pip install ortools
#!pip install dtale

     |████████████████████████████████| 32.7MB 118kB/s 


In [1]:
import pandas as pd
import numpy as np
import collections
from ortools.sat.python import cp_model
# Clone the entire repo.
!git clone -l -s https://github.com/mswastik/optimization.git cloned-repo
%cd cloned-repo

fatal: destination path 'cloned-repo' already exists and is not an empty directory.
/content/cloned-repo


In [2]:
#Importing Data
df=pd.read_excel('orders1.xlsx')
ma=pd.read_excel('item master.xlsx')
bu=pd.read_excel('item master.xlsx',sheet_name='Sheet2')
pa=pd.read_excel('item master.xlsx',sheet_name='packing')
 
# Formating Data
df = df[df['Line']=='Tmmthpkhti Limi']
df = df[df['Plant']==1024]
del df['Line']
df['due'] = df['Dispatch Date'] - df['SO Date'].min()
df['due'] = df['due'].dt.days
df['sodate'] = df['SO Date'] - df['SO Date'].min()
df['sodate'] = df['sodate'].dt.days
df.reset_index(inplace=True,drop=True)
df['Key']=df['SO'].astype('str')+df['FG Code']
df.drop(['Dispatch Date','SO Date','Description','Customer'],axis=1,inplace=True)
df['Order Qty'] = df[df['Order Qty']!=0]['Order Qty']
df['Order Qty'] = df['Order Qty']*1000
df=df[df['Key']!='17023287gd39000800iX']
 
 
# Creating Parameter Data
jobs =df['Key']
op =[1,2]  # Ignore as of now
mc = [1,2,3,4,5,6]
#horizon = max(df["due"])+900
due = df.set_index('Key')
dur=df[['Key','FG Code','Order Qty']]
dur=dur.merge(pa[["FG Code","Line","Output"]], on="FG Code")
dur['Output']=dur['Output']*100
dur['duration']=dur['Order Qty']/dur['Output']
dur['duration'] = dur['duration'].astype('float64').round(decimals = 2)
dur['operation'] = 2
budf=df.merge(ma[["FG Code","Bulk Code","Case Size","Pack Wt(g)"]],on='FG Code',how='left')
budf['Qty']=budf['Order Qty']*budf['Case Size']*budf['Pack Wt(g)']/1000000
bdur=budf.merge(bu[["Bulk Code", "Line","Output"]],on="Bulk Code",how="left")
bdur['Output'] = bdur['Output'].replace(0,1)
bdur['duration'] = bdur['Qty']/bdur['Output']
bdur['duration'] = bdur['duration'].fillna(0).astype('int')
bdur['operation'] = 1
bdur['Order Qty'] = bdur['Qty']
bdur.drop(['Plant','due','sodate','Case Size','Pack Wt(g)','SO','Qty','Bulk Code'],axis=1,inplace=True)
dur=dur.append(bdur)
dur.set_index(['Key','Line','operation'],inplace=True)
del dur['FG Code']
dur['Output'].replace(0,1,inplace=True)
dur['Order Qty']=dur['Order Qty'].fillna(0).astype('int')
dur['duration']=dur['duration'].fillna(0).astype('int')
dur['Output']=dur['Output'].fillna(0).astype('int')
horizon = sum(dur['duration'])
 
#Setup
setup = pd.DataFrame(columns=('Key1','Key2','setup'))
for i in df['Key']:
  for k in df['Key']:
    if df[df['Key']==i]["FG Code"].values[0]==df[df['Key']==k]["FG Code"].values[0]:
      f = {'Key1':[i],"Key2":[k],"setup":[0]}
    else:
      f = {'Key1':[i],"Key2":[k],"setup":[4]}
    f = pd.DataFrame(data=f)
    setup = pd.concat([setup,f])
setup.set_index(['Key1','Key2'],inplace=True)

In [4]:
# Initialise model
model = cp_model.CpModel()

# Creating variables to store data
task_type = collections.namedtuple('task_type', 'start end dur run1')
mtj = collections.defaultdict(list)
all_tasks = {}
 
for j in jobs:
  for o in op:
    for m in mc:
      suffix = '_%s_%i_%i' % (j,m,o)
      start = model.NewIntVar(0, horizon, "start"+suffix)
      end = model.NewIntVar(0, horizon, "end"+suffix)
      run1 = model.NewIntVar(0, dur['duration'].loc[(j,m,o)].item(), "run"+suffix)
      duration = model.NewIntervalVar(start, run1, end, "duration"+suffix)
      all_tasks[j, m,o] = task_type(start=start, end=end, dur= duration, run1=run1)
      mtj[o,m].append(duration)

# Selection variable to store whether machine is selected to process job
b={}
for o in op:
  for j in jobs:
    for m in mc:
      b[j,m,o] = model.NewBoolVar('selection')
      model.Add(all_tasks[j, m,o].run1>0).OnlyEnforceIf(b[j,m,o])
      model.Add(all_tasks[j, m,o].run1==0).OnlyEnforceIf(b[j,m,o].Not())

# Variable to store sequence of jobs
a={}
for o in op:
  for j in jobs:
    for m in mc:
      for i in jobs:
        if j != i:
          a[j,i,m,o] = model.NewBoolVar('sequence')
          model.Add(all_tasks[i, m,o].start>all_tasks[j, m,o].start).OnlyEnforceIf(a[j,i,m,o])
          model.Add(all_tasks[i, m,o].start<all_tasks[j, m,o].start).OnlyEnforceIf(a[j,i,m,o].Not())

# Constraint: A machine can process only 1 job at a time 
for o in op:
  for m in mc:
    model.AddNoOverlap(mtj[o,m])

# Constraint: Complete production of full order quantity
com = {}
for o in op:
  for j in jobs:
    tt= 0
    for m in mc:
      comp = all_tasks[j,m,o].run1*dur["Output"].loc[(j,m,o)]
      tt = tt + comp
    com[j] = tt

    #Change ORDER QTY FOR BULK
    model.Add(com[j]*10 >= dur.loc[(j,slice(None),o)]['Order Qty'].values[0])
  
# Constraint of Setup Time
for o in op:     
  for j in jobs:
    for m in mc:
      for i in jobs:
        if j != i:
          #model.AddBoolOr([a[j,i,m],a[i,j,m]])
          model.Add(all_tasks[i, m,o].start >= all_tasks[j, m,o].end + setup.loc[(j,i)].values[0]).OnlyEnforceIf(a[j,i,m,o]).OnlyEnforceIf(b[j,m,o]).OnlyEnforceIf(b[i,m,o])
          #model.Add(all_tasks[j, m].start >= all_tasks[i, m].end + setup.loc[(j,i)].values[0]).OnlyEnforceIf(a[,i,m].Not()).OnlyEnforceIf(b[j,m]).OnlyEnforceIf(b[i,m])

for j in jobs:
    for m in mc:
      minst=model.NewIntVar(0, horizon, "minst"+j)
      model.AddMinEquality(minst,[all_tasks[j,k,1].start for k in mc])
      model.Add(all_tasks[j, m,2].start >= minst+4).OnlyEnforceIf(b[j,m,2])
      model.Add(all_tasks[j, m,2].start <= minst+8).OnlyEnforceIf(b[j,m,2])
gg={}
for j in jobs:
  aa=model.NewIntVar(0, horizon, "endrun"+j)
  #aa[m]=all_tasks[j,m].end
  model.AddMaxEquality(aa,[all_tasks[j,m,2].end for m in mc])
  gg[j] = due['due'].loc[j]- aa

#Dummy variable for minimizing absolute value in objective function
mm={}
for j in jobs:
  kk = model.NewIntVar(0, horizon, 'dummy'+j)
  mm[j] = kk
  model.Add(gg[j]<=mm[j])
  model.Add(-gg[j]<=mm[j])
 
model.Minimize(sum([mm[j] for j in jobs]))
solver = cp_model.CpSolver()
solver.parameters.num_search_workers = 16
solver.Solve(model)
print(solver.StatusName(),solver.ObjectiveValue())

FEASIBLE 288.0


In [5]:
ff=pd.DataFrame()
for o in op:
  for m in mc:
    for j in jobs:
      if solver.Value(all_tasks[j,m,o].run1)>0:
        kk={'Job':j,'line':m,'operation':o,'start':solver.Value(all_tasks[j,m,o].start),
            'run':solver.Value(all_tasks[j,m,o].run1),'end':solver.Value(all_tasks[j,m,o].end),'due':due['due'].loc[j],'delay':solver.Value(gg[j])}
        ff=ff.append(kk,ignore_index=True)
''' 
!pip install -U plotly
import plotly.express as px
import datetime
fig=px.timeline(data_frame=ff, x_start=ff['start'].astype('datetime64[h]'),x_end=ff['end'].astype('datetime64[h]'),
                facet_row='line',y='Job',height=1200,width=1500)
fig.update_xaxes(dtick=14400000,tickformat="%d-%H")
fig.update_yaxes(autorange="reversed")
'''

' \n!pip install -U plotly\nimport plotly.express as px\nimport datetime\nfig=px.timeline(data_frame=ff, x_start=ff[\'start\'].astype(\'datetime64[h]\'),x_end=ff[\'end\'].astype(\'datetime64[h]\'),\n                facet_row=\'line\',y=\'Job\',height=1200,width=1500)\nfig.update_xaxes(dtick=14400000,tickformat="%d-%H")\nfig.update_yaxes(autorange="reversed")\n'

In [18]:
#!pip install -U bokeh
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource
output_notebook()

p = figure(y_range=ff['Job'].unique(),x_range=(0,300),plot_width=900, plot_height=550, title="Schedule")
p.hbar(y="Job", left='start', right='end', height=0.4, source=ff)
show(p)

In [ ]:
import plotly.graph_objects as go
from ipywidgets import widgets

In [ ]:
jo=widgets.SelectMultiple(options=ff['Job'].unique())
display(jo)
def gin(change):
  kjk=ff.loc[ff['Job']==jo.value[0]]
  #print(type(kjk))
  fig=px.timeline(data_frame=kjk, x_start=kjk['start'].astype('datetime64[h]'),x_end=kjk['end'].astype('datetime64[h]'),facet_row='line')
  fig.update_xaxes(dtick=14400000,tickformat="%d-%H")
  f=go.FigureWidget(data=fig)
  f.show()
jo.observe(gin,names='value')

SelectMultiple(options=('17023287gd29904300iX', '17023335gd28013000iX', '17023335gd28001700iX', '17023637gd280…

In [ ]:
ff

,Job,delay,due,end,line,operation,run,start
0,17023287gd29904300iX,0.0,80.0,59.0,1.0,1.0,1.0,58.0
1,17023335gd28013000iX,0.0,87.0,86.0,1.0,1.0,1.0,85.0
2,17023335gd28001700iX,0.0,87.0,135.0,1.0,1.0,1.0,134.0
3,17023637gd28001701iX,-61.0,87.0,1342.0,1.0,1.0,2.0,1340.0
4,17023440gd23010000gR,0.0,88.0,80.0,1.0,1.0,1.0,79.0
...,...,...,...,...,...,...,...,...
56,17023636gd280043C0iX,-42.0,82.0,124.0,6.0,2.0,30.0,94.0
57,17023335gd28001700iX,0.0,87.0,67.0,6.0,2.0,6.0,61.0
58,17023335gd30212200iX,0.0,87.0,9.0,6.0,2.0,5.0,4.0
59,17023439gd28311300iX,0.0,88.0,79.0,6.0,2.0,8.0,71.0


In [ ]:
#Exploring Result (click on last link to view result)
!pip install dtale
from dtale import show
import dtale.app as dtale_app
dtale_app.USE_COLAB = True
show(ff,ignore_duplicate=True)

     |████████████████████████████████| 7.6MB 2.8MB/s 
     |████████████████████████████████| 81kB 8.4MB/s 
     |████████████████████████████████| 194kB 48.0MB/s 
     |████████████████████████████████| 645kB 50.4MB/s 
     |████████████████████████████████| 71kB 8.7MB/s 
     |████████████████████████████████| 1.8MB 42.1MB/s 
     |████████████████████████████████| 952kB 53.5MB/s 
     |████████████████████████████████| 3.4MB 29.5MB/s 
     |████████████████████████████████| 194kB 48.1MB/s 
     |████████████████████████████████| 1.8MB 48.1MB/s 
     |████████████████████████████████| 358kB 41.5MB/s 
  Created wheel for dash: filename=dash-1.16.1-cp36-none-any.whl size=82320 sha256=bed2870b52b9843405193172647c98a763406af898a4f6be83895c3d3f6dc719
  Stored in directory: /root/.cache/pip/wheels/52/0c/98/a3842c450867b2aec8577ee1051bb94ad0f8fe62ed3f80c175
  Created wheel for dash-daq: filename=dash_daq-0.5.0-cp36-none-any.whl size=669704 sha256=3274070d497271b84f001102b45a2c17e19ecb21401

https://g94vhw19i5q-496ff2e9c6d22116-40000-colab.googleusercontent.com/dtale/main/1

In [ ]:
print(model.ModelStats())

Optimization model '':
#Variables: 2040 (17 in objective)
 - 1734 in [0,1]
 - 5 in [0,15]
 - 5 in [0,36]
 - 1 in [0,66]
 - 5 in [0,75]
 - 5 in [0,93]
 - 5 in [0,108]
 - 5 in [0,133]
 - 1 in [0,162]
 - 2 in [0,168]
 - 3 in [0,189]
 - 1 in [0,213]
 - 2 in [0,216]
 - 5 in [0,233]
 - 3 in [0,243]
 - 1 in [0,300]
 - 2 in [0,320]
 - 2 in [0,400]
 - 8 in [0,450]
 - 3 in [0,451]
 - 1 in [0,462]
 - 1 in [0,497]
 - 2 in [0,693]
 - 2 in [0,746]
 - 1 in [0,940]
 - 3 in [0,978]
 - 1 in [0,995]
 - 3 in [0,1054]
 - 6 in [0,1066]
 - 2 in [0,1493]
 - 5 in [0,2000]
 - 3 in [0,2108]
 - 1 in [0,2160]
 - 1 in [0,4500]
 - 1 in [0,5600]
 - 1 in [0,8000]
 - 1 in [0,8640]
 - 1 in [0,14000]
 - 1 in [0,15120]
 - 1 in [0,36000]
 - 204 in [0,1442570]
#kInterval: 102
#kLinear1: 204 (#enforced: 204)
#kLinear2: 4896 (#enforced: 4896)
#kLinearN: 17
#kNoOverlap: 6


In [ ]:
import plotly.io as pio
pio.renderers.default = "colab"